In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from anchor.visualize import MODALITY_TO_COLOR, MODALITY_ORDER, MODALITY_PALETTE
modality_order = MODALITY_ORDER

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = 'figures'

import flotilla
study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian',
# study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian_kmers_cisbp', 
# study = flotilla.embark('singlecell_pnm_figure2_modalities_rmdup', 
                        flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')
# study = flotilla.embark('singlecell_pnm_figure1_supplementary_post_splicing_filtering')

2017-01-14 10:39:58	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2017-01-14 10:39:58	Parsing datapackage to create a Study object
https://s3-us-west-2.amazonaws.com/flotilla-projects/ercc/ERCC_Controls.txt has not been downloaded before.


/home/obotvinnik/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Error loading species hg19 data: HTTP Error 404: Not Found

2017-01-14 10:40:46 	Initializing Study
2017-01-14 10:40:46 	Initializing Predictor configuration manager for Study
2017-01-14 10:40:46	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2017-01-14 10:40:46	Added ExtraTreesClassifier to default predictors
2017-01-14 10:40:46	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2017-01-14 10:40:46	Added ExtraTreesRegressor to default predictors
2017-01-14 10:40:46	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2017-01-14 10:40:46	Added GradientBoostingClassifier to default predictors
2017-01-14 10:40:46	Predictor GradientBoostingRegressor is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'>
2017-01-14 10:40:46	Added GradientBoostingRegressor to default predictors
2017-01-14 10:40:46 	Loading metadata
2017-01-14 10:40:46	Loading expression data
2017-0

In [2]:
study.splicing.minimum_samples

10

In [3]:
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=study.splicing.minimum_samples))

In [4]:
psi_filtered.head()

,isoform1=junction:chr10:100189400-100190327:-|isoform2=junction:chr10:100189647-100190327:-@exon:chr10:100189548-100189646:-@junction:chr10:100189400-100189547:-,isoform1=junction:chr10:100193849-100195391:-|isoform2=junction:chr10:100195172-100195391:-@exon:chr10:100195029-100195171:-@junction:chr10:100193849-100195028:-,isoform1=junction:chr10:101163392-101165512:-|isoform2=junction:chr10:101163632-101165512:-@exon:chr10:101163481-101163631:-@junction:chr10:101163392-101163480:-,isoform1=junction:chr10:101476219-101480743:-|isoform2=junction:chr10:101478258-101480743:-@exon:chr10:101478103-101478257:-@junction:chr10:101476219-101478102:-,isoform1=junction:chr10:101507148-101514285:+|isoform2=junction:chr10:101507148-101510125:+@exon:chr10:101510126-101510153:+@junction:chr10:101510154-101514285:+,isoform1=junction:chr10:101507148-101515381:+|isoform2=junction:chr10:101507148-101514285:+@exon:chr10:101514286-101514391:+@junction:chr10:101514392-101515381:+,isoform1=junction:chr10:101510154-101514285:+|isoform2=junction:chr10:101510154-101513352:+@novel_exon:chr10:101513353-101513445:+@junction:chr10:101513446-101514285:+,isoform1=junction:chr10:101995522-101997778:-|isoform2=junction:chr10:101996727-101997778:-@exon:chr10:101996607-101996726:-@junction:chr10:101995522-101996606:-,isoform1=junction:chr10:101996727-102003454:-|isoform2=junction:chr10:101997989-102003454:-@exon:chr10:101997779-101997988:-@junction:chr10:101996727-101997778:-,isoform1=junction:chr10:102006693-102013177:-|isoform2=junction:chr10:102010090-102013177:-@exon:chr10:102010005-102010089:-@junction:chr10:102006693-102010004:-,...,isoform1=junction:chrY:21897637-21903203:-|isoform2=junction:chrY:21901549-21903203:-@exon:chrY:21901414-21901548:-@junction:chrY:21897637-21901413:-,isoform1=junction:chrY:21901549-21903620:-|isoform2=junction:chrY:21903375-21903620:-@exon:chrY:21903204-21903374:-@junction:chrY:21901549-21903203:-,isoform1=junction:chrY:22744580-22749909:+|isoform2=junction:chrY:22744580-22746359:+@exon:chrY:22746360-22746410:+@junction:chrY:22746411-22749909:+,isoform1=junction:chrY:2709669-2712117:+|isoform2=junction:chrY:2709669-2710205:+@exon:chrY:2710206-2710283:+@junction:chrY:2710284-2712117:+,isoform1=junction:chrY:2710284-2713686:+|isoform2=junction:chrY:2710284-2712117:+@exon:chrY:2712118-2712298:+@junction:chrY:2712299-2713686:+,isoform1=junction:chrY:2712299-2722640:+|isoform2=junction:chrY:2712299-2713686:+@exon:chrY:2713687-2713784:+@junction:chrY:2713785-2722640:+,isoform1=junction:chrY:2713785-2722640:+|isoform2=junction:chrY:2713785-2721709:+@novel_exon:chrY:2721710-2721732:+@junction:chrY:2721733-2722640:+,isoform1=junction:chrY:2713785-2733128:+|isoform2=junction:chrY:2713785-2722640:+@exon:chrY:2722641-2722812:+@junction:chrY:2722813-2733128:+,isoform1=junction:chrY:2722813-2734833:+|isoform2=junction:chrY:2722813-2733128:+@exon:chrY:2733129-2733286:+@junction:chrY:2733287-2734833:+,isoform1=junction:chrY:2822039-2843135:+|isoform2=junction:chrY:2822039-2829114:+@exon:chrY:2829115-2829687:+@junction:chrY:2829688-2843135:+
CVN_01,NaN,NaN,1.0,NaN,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN
CVN_02,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.980583,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN
CVN_05,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,1.0,1.0,1.0,0.0,1.0,1.0,NaN


## Calculate *voyage* space via `bon voyage`

In [5]:
%%time

from bonvoyage import Waypoints

ws = Waypoints()

waypoints = psi_filtered.groupby(study.sample_id_to_phenotype).apply(
    lambda x: ws.fit_transform(x.dropna(how='all', axis=1)))

CPU times: user 15.9 s, sys: 546 ms, total: 16.4 s
Wall time: 16.2 s


In [6]:
waypoints.max()

0    1.0
1    1.0
dtype: float64

In [7]:
pd.DataFrame(ws.seed_data_transformed).max()

0    0.840891
1    0.999989
dtype: float64

In [8]:
transitions = study.phenotype_transitions + [('iPSC', 'MN')]
transitions

[(u'iPSC', u'NPC'), (u'NPC', u'MN'), ('iPSC', 'MN')]

In [9]:
from bonvoyage.voyages import Voyages

v = Voyages()

voyages = v.voyages(waypoints, transitions)
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,isoform1=junction:chr10:101163392-101165512:-|...,0.000000,0.033333,0.033333,iPSC,NPC,$\nwarrow$,iPSC-NPC
1,isoform1=junction:chr10:101476219-101480743:-|...,0.166667,-0.041667,0.171796,iPSC,NPC,$\searrow$,iPSC-NPC
2,isoform1=junction:chr10:101507148-101514285:+|...,0.291498,-0.089069,0.304802,iPSC,NPC,$\searrow$,iPSC-NPC
3,isoform1=junction:chr10:101510154-101514285:+|...,0.026316,-0.026316,0.037216,iPSC,NPC,$\searrow$,iPSC-NPC
4,isoform1=junction:chr10:101995522-101997778:-|...,0.181818,-0.150568,0.236069,iPSC,NPC,$\searrow$,iPSC-NPC


In [10]:
voyages['transition'] = voyages.group1 + '-' + voyages.group2
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,isoform1=junction:chr10:101163392-101165512:-|...,0.000000,0.033333,0.033333,iPSC,NPC,$\nwarrow$,iPSC-NPC
1,isoform1=junction:chr10:101476219-101480743:-|...,0.166667,-0.041667,0.171796,iPSC,NPC,$\searrow$,iPSC-NPC
2,isoform1=junction:chr10:101507148-101514285:+|...,0.291498,-0.089069,0.304802,iPSC,NPC,$\searrow$,iPSC-NPC
3,isoform1=junction:chr10:101510154-101514285:+|...,0.026316,-0.026316,0.037216,iPSC,NPC,$\searrow$,iPSC-NPC
4,isoform1=junction:chr10:101995522-101997778:-|...,0.181818,-0.150568,0.236069,iPSC,NPC,$\searrow$,iPSC-NPC


In [11]:
voyages.shape

(20401, 8)

### Add group1 and group2 modalities

In [12]:
study.supplemental.modalities_tidy.head()

,phenotype,event_id,modality
0,MN,isoform1=junction:chr10:101163392-101165512:-|...,included
1,MN,isoform1=junction:chr10:101476219-101480743:-|...,bimodal
2,MN,isoform1=junction:chr10:102114390-102120490:+|...,included
3,MN,isoform1=junction:chr10:102283687-102286731:-|...,included
4,MN,isoform1=junction:chr10:102746733-102747069:-|...,bimodal


In [13]:
groups = 'group1', 'group2'
voyages_modalities = voyages.copy()

for group in groups:
    voyages_modalities = voyages_modalities.merge(study.supplemental.modalities_tidy, 
                                       left_on=[group, 'event_id'], 
                                       right_on=['phenotype', 'event_id'], copy=False)
    print(voyages_modalities.shape)
    voyages_modalities = voyages_modalities.drop(['phenotype'], axis=1)
    voyages_modalities = voyages_modalities.rename(columns={'modality': '{}_modality'.format(group)})
print(voyages_modalities.shape)
voyages_modalities.head()

(20401, 10)
(20401, 11)
(20401, 10)


,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition,group1_modality,group2_modality
0,isoform1=junction:chr10:101163392-101165512:-|...,0.000000,0.033333,0.033333,iPSC,NPC,$\nwarrow$,iPSC-NPC,included,included
1,isoform1=junction:chr10:101163392-101165512:-|...,0.000000,0.033333,0.033333,iPSC,MN,$\nwarrow$,iPSC-MN,included,included
2,isoform1=junction:chr10:101163392-101165512:-|...,0.000000,0.000000,0.000000,NPC,MN,NaN,NPC-MN,included,included
3,isoform1=junction:chr10:101476219-101480743:-|...,0.166667,-0.041667,0.171796,iPSC,NPC,$\searrow$,iPSC-NPC,included,bimodal
4,isoform1=junction:chr10:101476219-101480743:-|...,0.052632,0.072368,0.089483,iPSC,MN,$\nearrow$,iPSC-MN,included,bimodal


## Add the data to the study and save it

In [14]:
study.supplemental.waypoints = waypoints
study.supplemental.voyages = voyages_modalities

In [15]:
study.save('singlecell_pnm_figure5_voyages', flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')

Wrote datapackage to /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_figure5_voyages/datapackage.json


/home/obotvinnik/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['ensembl_id', 'event_location', 'exon1', 'exon2', 'exon3', 'exon4', 'gene_id', 'gene_name', 'gene_status', 'gene_type', 'havana_gene', 'incompatible_junctions', 'intron_location', 'junction12', 'junction13', 'junction23', 'junction24', 'junction34', 'junctions', 'strand', 'tag', 'transcript_id', 'transcript_name', 'transcript_status', 'transcript_type']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [21]:
# ll /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_figure4_voyages

In [22]:
# ! md5sum /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure4_voyages/*csv